<a href="https://colab.research.google.com/github/fdsgusmao/projeto-final/blob/main/Projeto_Final_Workshop_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download do corpus

In [4]:
import pandas as pd
import numpy as np
import glob

In [5]:
!apt install subversion

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2 subversion
0 upgraded, 6 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,355 kB of archives.
After this operation, 10.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libapr1 amd64 1.6.5-1ubuntu1 [91.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libaprutil1 amd64 1.6.1-4ubuntu2.1 [84.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libserf-1-1 amd64 1.3.9-8build1 [45.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libutf8proc2 amd64 2.5.0-1 [50.0 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-

In [6]:
!svn checkout https://github.com/roneysco/Fake.br-Corpus/

Streaming output truncated to the last 5000 lines.
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2982.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2983.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2984.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2985.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2986.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2987.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2988.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2989.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/299.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2990.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2991.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2992.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2993.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2994.txt
A    Fake.br-Corpus/trunk/size_normalized_texts/fake/2995.txt
A    Fake.br-Corpus/

## Estruturando o dataset

In [7]:
# Pegando todos os nomes dos arquivos txt
fake_folder_path = '/content/Fake.br-Corpus/trunk/full_texts/fake'
true_folder_path = '/content/Fake.br-Corpus/trunk/full_texts/true'
true_file_list = glob.glob(true_folder_path + "/*.txt")
fake_file_list = glob.glob(fake_folder_path + "/*.txt")

In [8]:
true_news_list = []
for i in range(0,len(true_file_list)):
  # Abrindo todos os txt's
  with open(true_file_list[i], 'r') as file:
      # Tratando o txt e colocando numa lista
      data = file.read().replace('\n', '')
      true_news_list.append(data)

fake_news_list = []
for i in range(0,len(fake_file_list)):
  # Abrindo todos os txt's
  with open(fake_file_list[i], 'r') as file:
      # Tratando o txt e colocando numa lista
      data = file.read().replace('\n', '')
      fake_news_list.append(data)

In [9]:
# Transformando as strings de notícias em um DataFrame do pandas
df1 = pd.DataFrame()
df1['news'] = true_news_list
df1['fake'] = 0

df2 = pd.DataFrame()
df2['news'] = fake_news_list
df2['fake'] = 1

# Concatenando os DataFrames de fake e real news
main_df = pd.DataFrame()
main_df = main_df.append(df1, ignore_index = True)
main_df = main_df.append(df2, ignore_index = True)

<ipython-input-9-75069d4d4aa4>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(df1, ignore_index = True)
<ipython-input-9-75069d4d4aa4>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append(df2, ignore_index = True)


## Preprocessamento

In [12]:
# Importando o spacy e carregando o pipeline para fazer o preprocessamento
import spacy
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python -m spacy download pt_core_news_sm
nlp = spacy.load('pt_core_news_sm', exclude=['tok2vec', 'parser', 'lemmatizer', 'attribute_ruler', 'ner'])

2023-05-31 18:05:34.322933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 18.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [13]:
# Definindo a função de preprocessamento
def preprocessed(text):
  doc = nlp(text)
  preprocessedText = [t for t in doc if t.is_punct == 0 if t.is_stop == 0]
  preprocessedText = ' '.join(str(e) for e in preprocessedText)
  return preprocessedText

In [14]:
# Aplicando a função de preprocessamento em todas as notícias do DataFrame
main_df['preprocessed_news'] = main_df['news'].apply(preprocessed)

## Setup de treinamento

In [15]:
# Instalando o pacote transform
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.6 MB/s eta 0:00:00


In [16]:
# Imports que serão utilizados no treinamento
import torch
from torch import optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

In [17]:
# Separação do dataset em treino, teste e validação
X_train,X_test,y_train,y_test=train_test_split(main_df.preprocessed_news,main_df.fake,test_size=0.2,random_state=42)
X_train,X_validate,y_train,y_validate=train_test_split(X_train,y_train,test_size=0.1,random_state=42)

trainset = pd.concat([X_train,y_train],axis=1).to_dict('records')
testset = pd.concat([X_test,y_test],axis=1).to_dict('records')
validateset = pd.concat([X_validate,y_validate],axis=1).to_dict('records')

from torch.utils.data import DataLoader
batch_size = 16

traindata = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testdata = DataLoader(testset, batch_size=batch_size, shuffle=True)
validatedata = DataLoader(validateset, batch_size=batch_size, shuffle=True)

In [18]:
# Configurações de treino
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
nclasses = 2
nepochs = 3
batch_status = 32
learning_rate = 5e-5

max_length = 180

In [19]:
# Definindo a função de avaliação
def evaluate(model, testdata):
  model.eval()
  y_real, y_pred = [], []
  for batch_idx, inp in enumerate(testdata):
    texts, labels = inp['preprocessed_news'], inp['fake']
    
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
    output = model(**inputs)
                
    pred_labels = torch.argmax(output.logits, 1)
    
    y_real.extend(labels.tolist())
    y_pred.extend(pred_labels.tolist())
    
    if (batch_idx+1) % batch_status == 0:
      print('EVALUATION Progress:', round(batch_idx / len(testdata), 2), batch_idx)
  
  print(classification_report(y_real, y_pred, labels=[0, 1], target_names=['True', 'Fake']))
  f1 = f1_score(y_real, y_pred, average='weighted')
  acc = accuracy_score(y_real, y_pred)
  return f1, acc

In [20]:
# Definindo função de treinamento
def train(model, tokenizer):
  for epoch in range(nepochs):
    model.train()
    losses = []
    for batch_idx, inp in enumerate(traindata):
      texts, labels = inp['preprocessed_news'], inp['fake']

      # Classificação
      inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
      output = model(**inputs, labels=labels.to(device))

      # Cálculo do Loss
      loss = output.loss
      losses.append(float(loss))

      # Backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      # Display de progresso
      if (batch_idx+1) % batch_status == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTotal Loss: {:.6f}'.format(epoch, \
          batch_idx+1, len(traindata), 100. * batch_idx / len(traindata), 
          float(loss), round(sum(losses) / len(losses), 5)))
    
    f1, acc = evaluate(model, testdata)
    print('F1: ', f1, 'Accuracy: ', acc)

# Treinando o BERT

In [21]:
# Baixando o modelo
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=nclasses).to(device)

# Configurando o optimizador
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
train(model, tokenizer)

Train Epoch: 0 [32/324 (10%)]	Loss: 0.688720	Total Loss: 0.635960
Train Epoch: 0 [64/324 (19%)]	Loss: 0.382817	Total Loss: 0.542030
Train Epoch: 0 [96/324 (29%)]	Loss: 0.253577	Total Loss: 0.496880
Train Epoch: 0 [128/324 (39%)]	Loss: 0.391660	Total Loss: 0.461800
Train Epoch: 0 [160/324 (49%)]	Loss: 0.193060	Total Loss: 0.447690
Train Epoch: 0 [192/324 (59%)]	Loss: 0.286964	Total Loss: 0.440000
Train Epoch: 0 [224/324 (69%)]	Loss: 0.335636	Total Loss: 0.423500
Train Epoch: 0 [256/324 (79%)]	Loss: 0.291204	Total Loss: 0.418180
Train Epoch: 0 [288/324 (89%)]	Loss: 0.214125	Total Loss: 0.411250
Train Epoch: 0 [320/324 (98%)]	Loss: 0.314496	Total Loss: 0.410830
EVALUATION Progress: 0.34 31
EVALUATION Progress: 0.7 63
              precision    recall  f1-score   support

        True       0.76      0.98      0.85       718
        Fake       0.97      0.69      0.80       722

    accuracy                           0.83      1440
   macro avg       0.86      0.83      0.83      1440
weig

In [23]:
# Avaliando o modelo nas amostras de validação
evaluate(model, validatedata)

EVALUATION Progress: 0.86 31
              precision    recall  f1-score   support

        True       0.85      0.95      0.90       303
        Fake       0.93      0.82      0.87       273

    accuracy                           0.89       576
   macro avg       0.89      0.88      0.88       576
weighted avg       0.89      0.89      0.88       576



(0.88465576171875, 0.8854166666666666)

# Treinando o DistilBERT

In [27]:
# Baixando o modelo
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=nclasses).to(device)

# Configurando o optimizador
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [28]:
train(model, tokenizer)

Train Epoch: 0 [32/324 (10%)]	Loss: 0.669538	Total Loss: 0.663550
Train Epoch: 0 [64/324 (19%)]	Loss: 0.429908	Total Loss: 0.603710
Train Epoch: 0 [96/324 (29%)]	Loss: 0.559920	Total Loss: 0.558500
Train Epoch: 0 [128/324 (39%)]	Loss: 0.205279	Total Loss: 0.535610
Train Epoch: 0 [160/324 (49%)]	Loss: 0.297430	Total Loss: 0.508060
Train Epoch: 0 [192/324 (59%)]	Loss: 0.435040	Total Loss: 0.489630
Train Epoch: 0 [224/324 (69%)]	Loss: 0.308437	Total Loss: 0.469180
Train Epoch: 0 [256/324 (79%)]	Loss: 0.209978	Total Loss: 0.450850
Train Epoch: 0 [288/324 (89%)]	Loss: 0.446330	Total Loss: 0.441900
Train Epoch: 0 [320/324 (98%)]	Loss: 0.205171	Total Loss: 0.433740
EVALUATION Progress: 0.34 31
EVALUATION Progress: 0.7 63
              precision    recall  f1-score   support

        True       0.89      0.88      0.88       718
        Fake       0.88      0.89      0.89       722

    accuracy                           0.88      1440
   macro avg       0.88      0.88      0.88      1440
weig

In [29]:
# Avaliando o modelo nas amostras de validação
evaluate(model, validatedata)

EVALUATION Progress: 0.86 31
              precision    recall  f1-score   support

        True       0.92      0.91      0.91       303
        Fake       0.90      0.91      0.91       273

    accuracy                           0.91       576
   macro avg       0.91      0.91      0.91       576
weighted avg       0.91      0.91      0.91       576



(0.9097374961731199, 0.9097222222222222)

# Treinando o BERTimbau

In [24]:
# Baixando o modelo
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
model = AutoModelForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=nclasses).to(device)

# Configurando o optimizador
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [25]:
train(model, tokenizer)

Train Epoch: 0 [32/324 (10%)]	Loss: 0.236719	Total Loss: 0.525580
Train Epoch: 0 [64/324 (19%)]	Loss: 0.451080	Total Loss: 0.405580
Train Epoch: 0 [96/324 (29%)]	Loss: 0.154869	Total Loss: 0.335470
Train Epoch: 0 [128/324 (39%)]	Loss: 0.132118	Total Loss: 0.299370
Train Epoch: 0 [160/324 (49%)]	Loss: 0.198932	Total Loss: 0.279800
Train Epoch: 0 [192/324 (59%)]	Loss: 0.073458	Total Loss: 0.262300
Train Epoch: 0 [224/324 (69%)]	Loss: 0.125231	Total Loss: 0.248670
Train Epoch: 0 [256/324 (79%)]	Loss: 0.066100	Total Loss: 0.235950
Train Epoch: 0 [288/324 (89%)]	Loss: 0.086992	Total Loss: 0.223540
Train Epoch: 0 [320/324 (98%)]	Loss: 0.195022	Total Loss: 0.217290
EVALUATION Progress: 0.34 31
EVALUATION Progress: 0.7 63
              precision    recall  f1-score   support

        True       0.96      0.97      0.97       718
        Fake       0.97      0.96      0.97       722

    accuracy                           0.97      1440
   macro avg       0.97      0.97      0.97      1440
weig

In [26]:
# Avaliando o modelo nas amostras de validação
evaluate(model,validatedata)

EVALUATION Progress: 0.86 31
              precision    recall  f1-score   support

        True       0.99      0.95      0.97       303
        Fake       0.94      0.99      0.97       273

    accuracy                           0.97       576
   macro avg       0.97      0.97      0.97       576
weighted avg       0.97      0.97      0.97       576



(0.9670358804920455, 0.9670138888888888)